In [ ]:
import torch
import torch.nn as nn

from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import transforms

import torchvision.models as models

In [ ]:
import pandas as pd

train_df = pd.read_csv("../input/landmark-recognition-2020/train.csv")

train_df.head()

In [ ]:
landmark_id = str(train_df.iloc[0].landmark_id)
individual_image = train_df.iloc[0].id + '.jpg'

print(len(train_df.iloc[0].id))
print(individual_image, landmark_id )

from PIL import Image
import matplotlib.pyplot as plt

img = Image.open('../input/landmark-recognition-2020/train/'+individual_image[0]+ "/" + individual_image[1] + "/" + individual_image[2]+"/" + individual_image)
plt.imshow(img)


In [ ]:
mean = (0.485, 0.456, 0.406)
std =  (0.229,0.225,0.224)
IMG_SIZE = 128
transformations = transforms.Compose([transforms.Resize((IMG_SIZE,IMG_SIZE),interpolation=Image.NEAREST),
                                      transforms.ToTensor(),
                                      transforms.Normalize(mean,std)
                                     ]
                                    )    

In [ ]:
base_directory = "../input/landmark-recognition-2020/"

class get_landmarks(Dataset):
    def __init__(self,training_csv_path="../input/landmark-recognition-2020/train.csv", training_image_path="../input/landmark-recognition-2020/train/", my_transform=transformations):
        self.training_df = pd.read_csv(training_csv_path)
        self.image_path = training_image_path
        self.transform = my_transform
        
    def __len__(self):
        return len(self.training_df)
    
    def __getitem__(self,index):
        
        individual_image = self.training_df.iloc[index].id + ".jpg"
        landmark_id = self.training_df.iloc[index].landmark_id
        full_path = self.image_path +individual_image[0]+ "/" + individual_image[1] + "/" + individual_image[2]+ "/" + individual_image
#         print(full_path)
        
        img = Image.open(full_path)
        
        if self.transform:
            img = self.transform(img)
        
        return img, landmark_id
        
        
        

In [ ]:
train_dataset = get_landmarks()

x, y = train_dataset[0]

In [ ]:
print(y)
plt.imshow(x.permute(1,2,0))

In [ ]:
train_dataloader = DataLoader(dataset=train_dataset,batch_size=64,shuffle=True )

In [ ]:
model = models.resnet18(pretrained=True)

In [ ]:
model

In [ ]:
len(train_df['landmark_id'].unique())

# what the hell!

In [ ]:
model.fc = nn.Linear(in_features=512, out_features=len(train_df['landmark_id'].unique()), bias=True)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
model = model.to(device=device)